# Time Series Analysis of Motor Collisions in Seattle

In [1]:
import pandas as pd
import numpy as np


pkgs = [pd, np]
for pkg in pkgs:
    print(f'{pkg.__name__}: {pkg.__version__}')

pandas: 0.24.2
numpy: 1.16.2
